# Save data to an SQLite database for delivery via Datasette

This notebook brings together all the processed data – book metadata, extracted placenames and sentences, and country records from Wikidata. It saves the data to an SQLite database that can be delivered via Datasette.

In [1]:
import pandas as pd
import re
from sqlite_utils import Database

In [2]:
# Load sentences
sentences = pd.read_csv("sentences.tsv", sep="\t", header=None, names=["book_id", "text", "sentence"], dtype={"book_id": str, "text": str})

In [3]:
sentences.head()

,book_id,text,sentence
0,00000003,Scotland,Scotland was the home of Robert Burns and Walt...
1,00000003,Scotland,"England, though less wild and picturesque than..."
2,00000003,Scotland,Scotland is a wilder country than Ireland.
3,00000003,Scotland,Scotland has moors of yellow broom and hill- s...
4,00000003,Congo,For Africa the order should be; mountains Рђћ ...


In [4]:
# Some normalising, remove 'the' from the beginning of refs
sentences["text"] = sentences["text"].apply(lambda x: re.sub(r"^the ", "", x))

In [5]:
sentences.shape

(1417801, 3)

In [6]:
# Make surre there's no duplicates
sentences.drop_duplicates(subset=["book_id", "text", "sentence"], inplace=True)

In [7]:
# remove non-alphanumeric characters from the beginning of the string
sentences["sentence"] = sentences["sentence"].apply(lambda x: re.sub(r"^[^a-zA-Z0-9]*", "", x))

In [8]:
# Try to clean up hyphenation
sentences["sentence"] = sentences["sentence"].apply(lambda x: re.sub(r"([a-z])-\s([a-z])", r"\1\2", x))

In [9]:
# remove extra spaces
sentences["sentence"] = sentences["sentence"].apply(lambda x: re.sub(r"\s+", r" ", x))

## Countries

In [10]:
# Load country data
countries = pd.read_csv("cleaned_countries.csv")

In [11]:
# Add an id column from the Wikidata url
countries.insert(0, "id", countries["country"].str.extract(r"\/(Q\d+)"))
countries.head()

,id,text,country,startDate,endDate,countryLabel,countryTypeLabel,lat,lon
0,Q408,AUS,http://www.wikidata.org/entity/Q408,1901-01-01T00:00:00Z,NaN,Australia,country,-28.00,137.000000
1,Q207521,Abyssinia,http://www.wikidata.org/entity/Q207521,NaN,1974-01-01T00:00:00Z,Ethiopian Empire,historical country,12.60,37.466667
2,Q4461035,Accad,http://www.wikidata.org/entity/Q4461035,-2333-01-01T00:00:00Z,-2153-01-01T00:00:00Z,Akkadian empire,historical country,33.10,44.100000
3,Q775550,Adamawa,http://www.wikidata.org/entity/Q775550,1809-01-01T00:00:00Z,1903-07-29T00:00:00Z,Adamawa Emirate,historical country,9.15,10.000000
4,Q889,Afghanistan,http://www.wikidata.org/entity/Q889,1709-01-01T00:00:00Z,NaN,Afghanistan,country,33.00,66.000000


In [12]:
country_ids = countries.copy()[["id", "text"]]

# Add country ids to sentences
sentences_with_country_ids = pd.merge(sentences, country_ids, how="left", left_on="text", right_on="text")

In [13]:
sentences_with_country_ids.columns = ["book_id", "country", "sentence", "country_id"]

In [14]:
# Get a list of countries that are mentioned in sentences
# Because we filtered sentences by various patterns, not all countries will now be included
ids_in_use = list(sentences_with_country_ids["country_id"].unique())

In [15]:
# Dedupe and reorganise countries
countries_deduped = countries.loc[countries["id"].isin(ids_in_use)][["id", "countryLabel", "country", "startDate", "endDate", "countryTypeLabel", "lat", "lon"]]
countries_deduped.drop_duplicates(subset=["id"], inplace=True)

countries_deduped.columns = ["id", "title", "url", "start_date", "end_date", "country_type", "lat", "lon"]

In [16]:
# get the book metadata
metadata = pd.read_csv("cleaned_meta.csv", dtype={"id": str, "text": str})
metadata.head()

,id,url,title,contributors,details,date,image_url
0,ltf90003732,http://www.loc.gov/item/ltf90003732/,"Art, society, and accomplishments at home and ...","Blackburn, R. Barry. [from old catalog]","Chicago, W. B. Conkey Co., 1895.",1895,https://tile.loc.gov/image-services/iiif/publi...
1,ltf90025186,http://www.loc.gov/item/ltf90025186/,"How to win; or, Sure secrets of success. Menta...","Anderson, L. H. (Lewis H.)","Chicago, National Hygienic Institute, 1893.",1893,https://tile.loc.gov/image-services/iiif/publi...
2,ltf90017756,http://www.loc.gov/item/ltf90017756/,The purpose of God.,"Dodge, Joseph Smith. [from old catalog]","Boston : Universalist Pub. House, 1894.",1894,https://tile.loc.gov/image-services/iiif/publi...
3,ltf90007547,http://www.loc.gov/item/ltf90007547/,La ruine du Talmud; eÌpitre adresseÌe au gra...,"Bettelheim, Bernard Jean, 1811-1869.","Smyrne, Impr. d'A. Damiano, 1840.",1840,https://tile.loc.gov/image-services/iiif/servi...
4,ltf90010477,http://www.loc.gov/item/ltf90010477/,Der Islam in neunzehnten Jahrhundert; eine cul...,"VÃ¡mbÃ©ry, Ãrmin, 1832-1913.","Leipzig, F.A. Brockhaus, 1875.",1875,https://tile.loc.gov/image-services/iiif/publi...


In [17]:
# Create an SQLite db
db = Database("datasette/loc-books.db", recreate=True)

In [18]:
# Add the metadata
db["books"].insert_all(metadata.to_dict(orient="records"), pk="id")

<Table books (id, url, title, contributors, details, date, image_url)>

In [19]:
# Add countries
db["countries"].insert_all(countries_deduped.to_dict(orient="records"), pk="id")

<Table countries (id, title, url, start_date, end_date, country_type, lat, lon)>

In [20]:
# Add sentences
db["sentences"].insert_all(sentences_with_country_ids.to_dict(orient="records"), foreign_keys=[("book_id", "books", "id"), ("country_id", "countries", "id")])

<Table sentences (book_id, country, sentence, country_id)>

In [21]:
# Add full text index on text fields
db["books"].enable_fts(["title"])
db["countries"].enable_fts(["title"])
db["sentences"].enable_fts(["sentence"])

<Table sentences (book_id, country, sentence, country_id)>